<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

_by Grayson Badgley, April 2022_

The first step in figuring out how much carbon is lost during a fire is
determining how much carbon was around in the first place. This notebook serves
as a record of where those data came from and the transformations (if any)
performed to arrive at pre-fire biomass estimates.

# Projects

We are aware of _six_ projects that have burned to date. Two of those
wildfire-induced reversals have been verified by CARB and compensated for by the
buffer pool. The four remaining projects are:

- ACR255: Confederated Tribes of Colville Reservation
- ACR260: Confederated Tribes of Warm Springs Reservation
- ACR273: Klamath East
- CAR1102: Montesol

Having the right carbon at the outset of the fire is especially important for
determining whether or not the project automatically terminates. Alternatively,
underestimating pre-fire carbon stocks makes estimates of the size of the
reversal conservative.


In [ ]:
import json

import fsspec
import pandas as pd

records = {}

In [ ]:
def grow_pools_forward(carbon_stocks, event_year):
    """Calculate median growth of carbon pools.

    Then apply that growth rate n times, where n is the difference between the year of the event
    and the year of the last OPDR
    """
    nyears = event_year - carbon_stocks.index.max()
    scaled = (
        1 + carbon_stocks.pct_change().median()
    ) ** nyears * carbon_stocks.iloc[-1]
    return scaled.round().astype(int).to_dict()

## ACR255

Multiple fires have burned through this project.

We'll use the carbon pools reported in the most recent OPDR (covering July 2016
to July 2017) and make no further adjustments. This makes our estimates of
carbon loss conservative, at the expense of preventing us from determining
whether the cummulative effects of fire have terminated the project. Making a
determination on termination will require a separate analysis.


In [ ]:
records["acr255"] = {
    "ifm-1": 45_781_013,
    "ifm-3": 5_529_953,
}  # rounded to nearest tCO2

## ACR260

The 2020
[Lionshead fire](https://www.opb.org/article/2021/08/30/wildfire-warm-springs-lionshead-prescribed-burning/)
burned through this project.

ACR260 has submitted four annual OPDRs to the
[ACR registry](https://acr2.apx.com/mymodule/reg/TabDocuments.asp?r=111&ad=Prpt&act=update&type=PRO&aProj=pub&tablename=doc&id1=260),
with the most recent spanning 2017/2018. Lionshead ignited
[August 16, 2020](https://inciweb.nwcg.gov/incident/7049/), two years later. To
make our estimates conservative, we'll just grow the project forward one year.


In [ ]:
event_year_less_one = 2019
raw_data = [
    (2015, 4_105_735, 829_223),
    (2016, 4_440_489, 927_324),
    (2017, 4_514_130, 914_989),
    (2018, 4_588_312, 894_781),
]
carbon_pools = pd.DataFrame(
    raw_data, columns=["year", "ifm-1", "ifm-3"]
).set_index("year")

records["acr260"] = grow_pools_forward(carbon_pools, event_year_less_one)

In [ ]:
records

## ACR273

The 2021 [Bootleg fire] burned through this project.

ACR273 has submitted five annual OPDRs to the
[ACR registry](https://acr2.apx.com/mymodule/reg/TabDocuments.asp?r=111&ad=Prpt&act=update&type=PRO&aProj=pub&tablename=doc&id1=273),
with the most recent spanning 2019-2020. Bootleg ignited
[July 12, 2021](https://inciweb.nwcg.gov/incident/7609/), about six months after
the most recent OPDR. To ensure a conservative estimate of loses, we'll just use
the 2020 biomass numbers


In [ ]:
event_year = 2021
raw_data = [
    (2016, 13_947_830, 334_352),
    (2017, 14_474_946, 334_352),
    (2018, 14_937_966, 334_352),
    (2019, 15_119_235, 513_299),
    (2020, 15_513_892, 513_299),
]
carbon_pools = pd.DataFrame(
    raw_data, columns=["year", "ifm-1", "ifm-3"]
).set_index("year")

records["acr273"] = {"ifm-1": raw_data[-1][1], "ifm-3": raw_data[-1][2]}

In [ ]:
records

## CAR1102

The 2020 Hennessey and Glass fires (i.e., two separate fires) burned through
this project.

CAR1102 has submitted four annual OPDRs to the
[CAR registery](https://thereserve2.apx.com/mymodule/reg/TabDocuments.asp?r=111&ad=Prpt&act=update&type=PRO&aProj=pub&tablename=doc&id1=1102),
with the most recent updating carbon pools through the 2019 growing season. The
Hennessey and Glass fires ignited in
[mid August](https://en.wikipedia.org/wiki/LNU_Lightning_Complex_fires) and
[late September](https://en.wikipedia.org/wiki/Glass_Fire).

This project also exhibits irregular step-changes in IFM-3. We'll double check
to make sure IFM-3 is set equal to the last reported value (as opposed to
growing forward in an unrealistic manner).


In [ ]:
event_year = 2020
raw_data = [
    (2016, 828_840, 12_978),
    (2017, 836_414, 12_978),
    (2018, 843_988, 12_978),
    (2019, 852_198, 19_204),
]
carbon_pools = pd.DataFrame(
    raw_data, columns=["year", "ifm-1", "ifm-3"]
).set_index("year")

records["car1102"] = {"ifm-1": raw_data[-1][1], "ifm-3": raw_data[-1][2]}

In [ ]:
records

In [ ]:
fn = "adjusted_prefire_carbon_stocks.json"
out_dir = "gcs://carbonplan-buffer-analysis"
out_fn = "/".join([out_dir, fn])

with fsspec.open(out_fn, "w") as f:
    json.dump(records, f, indent=2)